In [ ]:
from IPython.display import clear_output

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import json
import re
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
!pip install transformers kss hanja
clear_output()

In [ ]:
import kss
import hanja
from transformers import Trainer, TrainingArguments

[Korean Sentence Splitter]: Initializing Kss...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## keyword module

In [ ]:
# 토크나이저 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

%cd /content
clear_output()

In [ ]:
# textrank 설치
%cd /content/
!git clone --recursive https://github.com/lovit/textrank
import sys
sys.path.insert(0,'/content/textrank')
clear_output()

In [ ]:
from textrank import KeywordSummarizer
from textrank import KeysentenceSummarizer
from konlpy.tag import Mecab

In [ ]:
# pos 태깅 NN(명사) 단어 추출
def pos_nn_words(sent):
    mecab = Mecab()
    words = mecab.pos(sent, join=True)
    words = [w for w in words if ('/NN' in w and list(w)[1]!='/')]
    
    words = [w.split('/')[0] for w in words]
    return words

def summarizer_text(texts, n):
    summarizer = KeysentenceSummarizer(
        tokenize = pos_nn_words,
        min_sim = 0.3,
        verbose = False
    )
    sent = []
    keysents = summarizer.summarize(texts, topk=n)

    for _, _, a in keysents:
        sent.append(a)
    
    final_sent = []
    
    for i in texts:
        if i in sent:
            final_sent.append(i)
    
    return final_sent

def keywords_by_textrank(text, n):
    keyword_extractor = KeywordSummarizer(
    tokenize = pos_nn_words,
    window = -1,
    verbose = False,
    min_count=1) # FOR working with extremely short text.

    textrank_keywords = keyword_extractor.summarize([text], n)

    return textrank_keywords

## Text preprocessing
- 

In [ ]:
# csv에서 list of strings로 변환
df = pd.read_csv('/content/drive/MyDrive/test/inter/interview.csv')
lines = pd.Series(df['text'])

In [ ]:
# txt에서 list of strings로 변환
with open('/content/drive/MyDrive/interview1.txt') as f:
    lines = f.readlines()
lines = pd.Series(lines)

In [ ]:
# txt에서 list of strings로 변환
with open('/content/drive/MyDrive/interview2.txt') as f:
    lines = f.readlines()
lines = pd.Series(lines)

In [ ]:
# 줄바꿈, 리턴 지우기
lines = lines.apply(lambda x: x.replace('\r', '').replace('\n', ''))
lines = lines[lines!=''].reset_index(drop=True)

In [ ]:
# <, > 특수문자 지우기
lines = lines.apply(lambda x: re.sub('[<>]', '', x))

In [ ]:
# 3번 이상 반복되는 특수문자/문자/숫자는 마스킹 처리하기
lines = lines.apply(lambda x: re.sub('(([a-zA-Z0-9*○])\\2{2,})', '<unused0>', x))

In [ ]:
# 괄호 안 내용 및 괄호 지우기
lines = lines.apply(lambda x: re.sub(r'\([^)]*\)', '', x))

In [ ]:
# 특수문자 지우기
lines = lines.apply(lambda x: re.sub('[\'\"!‘’@#$`%^&*\{\}\|\;?]', '', x))

In [ ]:
# 한자 => 한글로 바꾸기
lines = lines.apply(lambda x: hanja.translate(x, 'substitution'))
# 혹시모르니 남은 한자 모두 지우기
lines = lines.apply(lambda x: re.sub('[一-龥]', '', x))

# 공백 두 개 된거 하나로
lines = lines.apply(lambda x: x.replace('  ', ' '))

In [ ]:
# 동그라미 문자 2개로 마스킹된 거 토크나이징
lines = lines.apply(lambda x: x.replace('oo', '<unused0>'))
lines = lines.apply(lambda x: x.replace('OO', '<unused0>'))
lines = lines.apply(lambda x: x.replace('ㅇㅇ', '<unused0>'))
lines = lines.apply(lambda x: x.replace('○○', '<unused0>'))

## 년도는 빼고 숫자 00 마스킹 어떻게 하는지 모르겠음..
# 기업명 목록이 모두 <unk>로 나오는데, 이걸 <unused0>으로 합치는 게 좋을 수도 있겠다.

In [ ]:
# 문장 양쪽 공백 제거
lines = lines.apply(lambda x: x.strip())

# Series to list
lines = lines.to_list()

In [ ]:
# 문장으로 분리
lines = kss.split_sentences(lines)

In [ ]:
new_lines = []
for i in lines:
    new_lines.append(" ".join(i))
    
new_lines[:5]

['서로 상생하고 기업과 직원이 서로를 존중할 수 있는지를 기준으로 회사를 고려합니다. 한미약품은 직원을 존중해주고 사람의 가치와 인격을 우선하는 곳이라고 판단되어 지원하게 되었습니다. 우리는 살아가면서 사람들이 누군가를 판단할 때 겉모습과 행동을 통해 섣부를 판단을 하고 프레임을 씌우며 선입견을 가집니다. 이러한 특징 때문에 우리가 진정으로 봐야 할 내면의 가치와 인격을 고려하지 않으며 자신의 섣부른 판단이 옳다고 착각을 하며 상대방에 대한 인식을 바꾸려 하지 않습니다. 하지만 제가 보았을 때, 한미약품이 특별한 이유는 사람의 내면의 가치를 우선 파악하고 상대방에 대한 편견과 선입견을 섣불리 가지지 않으려 하며 사람마다 자신 내면의 가치와 아름다움을 파악할 수 있게 도움을 주기 때문입니다. 따라서, 한미약품에서 함께 일하는 일원이 되고 싶습니다.',
 '인내심과,집념 성취욕이 강한 사람입니다. 관련 경험으로 캐나다의 초등학교 웅변대회에서 3등을 수상했던 적이 있습니다. 캐나다에서 12살 때 일 년동안 홈스테이 생활을 하며 현지인 학생들과 함께 학교를 다녔습니다. 연수생활 6개월 쯤, 학교에서 웅변대회를 열었고 총 5명에게 상을 수여하는 대회였습니다. 수많은 사람들이 보는 무대에 서서 한국 문화를 주제로 연설을 해보고 싶었습니다. 한국이라고 하면 대부분 사람들이 서울만 알고 있었습니다. 그래서 한국에 대해서 좀 더 알리고자 웅변대회 참여에 결심했습니다. 하지만 가장 큰 문제점은 영어 실력과 자신감이었기 때문에 학교 선생님들에게 대본과 말하기 실력에 대한 피드백을 지속적으로 받았습니다. 저는 웅변대회 열흘 전부터 하루에 10번 이상 연습했고 선생님들에게 제 노력을 어필한 것은 물론이고 대본 숙지의 완벽함 덕분에 3등을 했습니다. 또한 영어가 모국어가 아님에도 불구하고 당당함을 보였던 사례라고 교장선생님에게 칭찬을 얻었습니다. 이를 통해 배운 것은 목표 도달을 위한 끈기와 열정입니다.',
 '구매 직무 수행을 위해 보완할 점은 시장 동향 파악 역량입니다. 이와 약

In [ ]:
# lines_set = []

In [ ]:
# lines 합치기
lines_set += new_lines

print(lines_set[:5])
print(len(lines_set))

['중학교 이후로 계속하고 있는 육상을 통해서 노력이 가져오는 기량 향상에는 한계가 없고, 다음 목표는 실현되기 위해 존재하고 있다는 신념에 기초해서 저는 계속 달려왔습니다.', '막히더라도 다른 관점에서 사물을 봐라 저는 이러한 자세로 좌절할 듯한 상황이 되어도 항상 포기하지 않고 무슨 일이라도 적극적으로 노력해 왔습니다.', '최선을 지향하라, 하지막 최악에도 대비하라를 신조로, 저는 행동할 때에는 항상 신중함과 대담함이라는 상반되는 2가지를 염두에 두고 있습니다.', '저의 신조는 선입관을 가지지 않고 무슨 일에라도 도전한다, 항상 내가 모르는 것은 배워야 하며, 새로운 환경에 적응한다 이렇게 두가지입니다.', '저는 우리나라 속담에도 있는 구르는 돌은 이끼가 끼지 않는다를 좌우명으로 삼고 있으며, 단단히 땅에 다리를 붙이고 자기자신을 단련하고 있으면 반드시 싹이 나올 것이라는 신념을 갖고 있습니다.']
1098


In [ ]:
keywords = []
for line in new_lines:
    temp = keywords_by_textrank(line, 3)
    temp = str([x[0] for x in temp]).replace('[','').replace(']','').replace('\'', '')
    # textrank_keywords = ','.join(textrank_keywords)
    keywords.append(temp)
    
keywords[:5]

['사람, 가치, 판단', '학교, 웅변대회, 사람', '시장, 파악, 구매', '학생, 업무, 담당자', '연수, 프로그램, 학생']

In [ ]:
# keyword_set = []

In [ ]:
keyword_set += keywords

print(keyword_set[:5])
print(len(keyword_set))

['기초, 신념, 존재', '노력, 적극, 포기', '염두, 가지, 상반', '가지, 적응, 환경', '신념, 단련, 자신']
1098


In [ ]:
##### 최종 dictionary  ((keyword_set + lines_set))
data = dict()
for i, (key, text) in enumerate(zip(keyword_set, lines_set)):
    data[i] = [key], text
data.items()

dict_items([(0, (['기초, 신념, 존재'], '중학교 이후로 계속하고 있는 육상을 통해서 노력이 가져오는 기량 향상에는 한계가 없고, 다음 목표는 실현되기 위해 존재하고 있다는 신념에 기초해서 저는 계속 달려왔습니다.')), (1, (['노력, 적극, 포기'], '막히더라도 다른 관점에서 사물을 봐라 저는 이러한 자세로 좌절할 듯한 상황이 되어도 항상 포기하지 않고 무슨 일이라도 적극적으로 노력해 왔습니다.')), (2, (['염두, 가지, 상반'], '최선을 지향하라, 하지막 최악에도 대비하라를 신조로, 저는 행동할 때에는 항상 신중함과 대담함이라는 상반되는 2가지를 염두에 두고 있습니다.')), (3, (['가지, 적응, 환경'], '저의 신조는 선입관을 가지지 않고 무슨 일에라도 도전한다, 항상 내가 모르는 것은 배워야 하며, 새로운 환경에 적응한다 이렇게 두가지입니다.')), (4, (['신념, 단련, 자신'], '저는 우리나라 속담에도 있는 구르는 돌은 이끼가 끼지 않는다를 좌우명으로 삼고 있으며, 단단히 땅에 다리를 붙이고 자기자신을 단련하고 있으면 반드시 싹이 나올 것이라는 신념을 갖고 있습니다.')), (5, (['생각, 추구, 발전'], '언행일치와 항상 새로운 이상의 추구를 신조로 하고 있습니다. 한번 마음속으로 정한 것은 실행되어야 한다고 생각하며, 또 항상 새로운 일을 추구하지 않으면 자신의 발전은 없다고 생각합니다.')), (6, (['생각, 평가, 주위'], '저의 신조는 어느 정도는 자기 마음대로 해라입니다. 인간은 집단 안에서 규율을 지켜야 하지만, 반면 때로는 마음먹고 색다른 일을 하지 않으면 주위로부터 평가는 얻을 수 없다고 생각합니다.')), (7, (['역경, 발휘, 기반'], '저의 세일포인트는 역경에 휘말려 든 때일수록, 그 역경을 기반으로 하여 새로운 힘을 발휘할 수 있다는 것입니다.')), (8, (['자세, 배신, 기대'], '초지일관을 신조로 각 부문에 주역으로서, 저는 항상 책임있는 행동과 기대를 

In [ ]:
import pickle
with open('/content/drive/MyDrive/datatest.txt', 'wb') as fp:
    pickle.dump(data, fp)

In [ ]:
# with open("/content/drive/MyDrive/keywordtest.txt", "rb") as fp:   # Unpickling
#     lines = pickle.load(fp)

## 모델 / 토크나이저

In [ ]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
model.cuda()

clear_output()

In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token='<s>', eos_token='</s>', unk_token='<unk>',
    pad_token='<pad>', mask_token='<mask>', additional_special_tokens=['<unused0>', '<unused1>'])

clear_output()
# <unused0> for proper nouns
# <unused1> for sep token

## 유틸

In [ ]:
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler
import random

# MAXLEN = 1024
MAXLEN = 768

UNFREEZE_LAST_N = 6 #The last N layers to unfreeze for training
SEED = 2020
TRAIN_SIZE = 0.9

In [ ]:
import os
import torch
import torch.nn as nn
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

## 데이터셋 클래스

In [ ]:
class myDataset(Dataset):
    def __init__(self, data, tokenizer, randomize=True):

        text, keywords = [], []
        for k, v in data.items():
            keywords.append(v[0])
            text.append(v[1])

        self.text = text
        self.keywords = keywords
        self.randomize = randomize
        self.tokenizer = tokenizer
    
    @staticmethod
    def join_keywords(keywords, randomize=True):
        N = len(keywords)
    
        # random sampling and shuffle
        if randomize:
            keywords = keywords[0].split(', ')
            random.shuffle(keywords)
        return ','.join(keywords)

        #     M = random.choice(range(N+1)) # 데이터셋이 많아지면 그냥 이거 써야겠다..
        #     keywords = keywords[:M]
        #     random.shuffle(keywords)
        # return ','.join(keywords)

    def __len__(self):
        return len(self.text)

    def __getitem__(self, i):
        keywords = self.keywords[i].copy()
        kw = self.join_keywords(keywords, self.randomize)

        input = tokenizer.bos_token + kw + '<unused1>' + \
                self.text[i] + tokenizer.eos_token
        
        encodings_dict = tokenizer(input,
                                   truncation=True,
                                   max_length=MAXLEN,
                                   padding='max_length')
        
        input_ids = encodings_dict['input_ids']
        attention_mask = encodings_dict['attention_mask']
        
        return {'label': torch.tensor(input_ids).to('cuda'),
                'input_ids': torch.tensor(input_ids).to('cuda'), 
                'attention_mask': torch.tensor(attention_mask).to('cuda')}
                # label, input_ids 둘 다 같게 해놔도
                # 모델에 들어갈 때는 label이 자동으로 shift 된다.

In [ ]:
def split_data(data, S=TRAIN_SIZE):
    # shuffle ids
    ids = list(data.keys())
    random.shuffle(ids)

    # split into training and validation sets
    train_size = int(S * len(data))
    
    train_ids = ids[:train_size]
    val_ids = ids[train_size:]
    
    train_data = dict()
    for id in train_ids:
        train_data[id] = data[id]

    val_data = dict()
    for id in val_ids:
        val_data[id] = data[id]
    
    return train_data, val_data

## 모델 파라미터 프리징

In [ ]:
# freeze selective layers
# - Freeze all layers except last n:
for parameter in model.parameters():
    parameter.requires_grad = True

for i, m in enumerate(model.transformer.h):
    #Only un-freeze the last n transformer blocks
    if i+1 > 12 - UNFREEZE_LAST_N:
        for parameter in m.parameters():
            parameter.requires_grad = True 

for parameter in model.transformer.ln_f.parameters():
    parameter.requires_grad = True

for parameter in model.lm_head.parameters():
    parameter.requires_grad = True

In [ ]:
train_data, val_data = split_data(data)
train_dataset = myDataset(train_data, tokenizer)
val_dataset = myDataset(val_data, tokenizer, randomize=False)

f'There are {len(train_dataset) :,} samples for training, and {len(val_dataset) :,} samples for validation testing'

'There are 988 samples for training, and 110 samples for validation testing'

In [ ]:
training_args = TrainingArguments(
    output_dir="/content/",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=12,
    eval_steps=50, # Number of update steps between two evaluations.
    save_steps=200, # after # steps model is saved 
    warmup_steps=100,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    weight_decay=0.01,
    learning_rate=5e-4,
    adam_epsilon=1e-8,
)

#---------------------------------------------------#
trainer = Trainer(
    model=model,
    args=training_args,    
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 988
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 1235


Step,Training Loss
500,0.795500
1000,0.304500


Saving model checkpoint to /content/checkpoint-200
Configuration saved in /content/checkpoint-200/config.json
Model weights saved in /content/checkpoint-200/pytorch_model.bin
tokenizer config file saved in /content/checkpoint-200/tokenizer_config.json
Special tokens file saved in /content/checkpoint-200/special_tokens_map.json
Saving model checkpoint to /content/checkpoint-400
Configuration saved in /content/checkpoint-400/config.json
Model weights saved in /content/checkpoint-400/pytorch_model.bin
tokenizer config file saved in /content/checkpoint-400/tokenizer_config.json
Special tokens file saved in /content/checkpoint-400/special_tokens_map.json
Saving model checkpoint to /content/checkpoint-600
Configuration saved in /content/checkpoint-600/config.json
Model weights saved in /content/checkpoint-600/pytorch_model.bin
tokenizer config file saved in /content/checkpoint-600/tokenizer_config.json
Special tokens file saved in /content/checkpoint-600/special_tokens_map.json
Saving model 

TrainOutput(global_step=1235, training_loss=0.464570538910777, metrics={'train_runtime': 727.6568, 'train_samples_per_second': 6.789, 'train_steps_per_second': 1.697, 'total_flos': 1936173957120000.0, 'train_loss': 0.464570538910777, 'epoch': 5.0})

In [ ]:
trainer.save_model('/content/drive/MyDrive/keywordmodel2')

Saving model checkpoint to /content/drive/MyDrive/keywordmodel2
Configuration saved in /content/drive/MyDrive/keywordmodel2/config.json
Model weights saved in /content/drive/MyDrive/keywordmodel2/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/keywordmodel2/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/keywordmodel2/special_tokens_map.json


In [ ]:
# # cache clear
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
# # example
# model2 = GPT2LMHeadModel.from_pretrained('/content/checkpoint-400')
# model2.to('cuda')

In [ ]:
# tokenizer2 = PreTrainedTokenizerFast.from_pretrained('/content/checkpoint-400')

In [ ]:
keywords = ['성실,열정,도전']
kw = myDataset.join_keywords(keywords, randomize=False)

prompt = tokenizer.bos_token + kw + '<unused1>'
         
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to('cuda')

model2.eval();

In [ ]:
# Top-p (nucleus) text generation (10 samples):
sample_outputs = model.generate(generated, 
                                do_sample=True,   
                                min_length=30,
                                max_length=MAXLEN,
                                top_k=50,                                 
                                top_p=0.95,        
                                # temperature=0.9,
                                repetition_penalty=2.0,
                                num_return_sequences=3
                                )

for i, sample_output in enumerate(sample_outputs):
    text = tokenizer.decode(sample_output, skip_special_tokens=True)
    a = len(','.join(keywords))
    print("{}: {}\n\n".format(i+1,  text[a:]))

1:  저는 부모님께서 항상 진취적인 기상을 가지고 미래를 준비하시며 이만큼은 스스로에게 주어진 것을 보다 성실하게 자신의 발전을 인정해 주셨습니다. 그래서 어려움을 극복하더라도 내가 먼저 혁신적인 결과를 보여시던 아버지께서는 언제나 활력으신 아버지와 함께 밝은 가정을 꾸리실 수 있도록 가르쳐주십니다.


2:  21세기 비즈니스 화두는 핵심역량, 선택과 집중이라 생각해봅니다. 앞으로의 미래는 예측 정확성과 가능성이 그 무엇보다 마케팅을 잘 아우르는 것에 있습니다.


3:  저는 오래전부터 준비해 오던 발전가능 문제를 끊임없이 모색했습니다. 다양한 분야에 대한 지식과 경험을 통해 오늘보다 나은 미래를 향해 노력하는 자세를 견지하고, 토론동아리에서 사람들과 대화를 하며 한 입니다. 항상 현재 가지고 있는 벽을 깨, 어떤 고난이더라도 매 순간 최선을 다하여 목표를 달성하는 데 앞장서겠습니다


